In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
import requests
from PIL import Image
from io import BytesIO
import matplotlib.pyplot as plt

# ------------------------------------------------
# 1. Load Dataset
# ------------------------------------------------
df = pd.read_csv(r"C:\Users\Asus\Downloads\dataset\movie dataset\movies dataset 480.csv", encoding="latin1")

# ------------------------------------------------
# 2. Encode Users and Movies
# ------------------------------------------------
user_encoder = {u: i for i, u in enumerate(df['userId'].unique())}
movie_encoder = {m: i for i, m in enumerate(df['movieId'].unique())}
user_decoder = {i: u for u, i in user_encoder.items()}
movie_decoder = {i: m for m, i in movie_encoder.items()}

df['user'] = df['userId'].map(user_encoder)
df['movie'] = df['movieId'].map(movie_encoder)

# ------------------------------------------------
# 3. Create User-Item Matrix
# ------------------------------------------------
num_users = len(user_encoder)
num_movies = len(movie_encoder)

R = np.zeros((num_users, num_movies))
for row in df.itertuples():
    R[row.user, row.movie] = row.rating

# ------------------------------------------------
# 4. Train-Test Split
# ------------------------------------------------
train_data, test_data = train_test_split(df, test_size=0.2, random_state=42)

R_train = np.zeros((num_users, num_movies))
for row in train_data.itertuples():
    R_train[row.user, row.movie] = row.rating

# ------------------------------------------------
# 5. Collaborative Filtering (Mean Imputation + SVD)
# ------------------------------------------------
overall_mean = np.mean(train_data['rating'])
user_mean = train_data.groupby('user')['rating'].mean()
movie_mean = train_data.groupby('movie')['rating'].mean()

def predict_rating(user, movie):
    if user in user_mean and movie in movie_mean:
        return (overall_mean + user_mean[user] + movie_mean[movie]) / 3
    else:
        return overall_mean

predictions = []
actuals = []

for row in test_data.itertuples():
    pred = predict_rating(row.user, row.movie)
    predictions.append(pred)
    actuals.append(row.rating)

rmse = np.sqrt(mean_squared_error(actuals, predictions))
accuracy = 100 - (rmse / 5 * 100)  # normalize RMSE into %
print(f"\n🎯 Model Accuracy (Overall): {accuracy:.2f}%\n")

# ------------------------------------------------
# 6. Fetch Poster + IMDb link from OMDb API
# ------------------------------------------------
OMDB_API_KEY = "41c5d7ea"

def fetch_movie_details(movie_name):
    url = f"http://www.omdbapi.com/?t={movie_name}&apikey={OMDB_API_KEY}"
    response = requests.get(url)

    if response.status_code == 200:
        data = response.json()
        if data.get("Response") == "True":
            poster_url = data.get("Poster", None)
            imdb_id = data.get("imdbID", "")
            imdb_link = f"https://www.imdb.com/title/{imdb_id}/"
            return poster_url, imdb_link
    return None, None

# ------------------------------------------------
# 7. Recommendation Function (All mood-matching movies)
# ------------------------------------------------
def recommend_movies_all(user_id, mood, df):
    user = user_encoder[user_id]
    user_movies = df[df['userId'] == user_id]

    # Exact mood match
    mood_movies = df[df['mood'].str.lower() == mood.lower()]
    # Relax mood filter if no exact match
    if len(mood_movies) == 0:
        mood_movies = df[df['mood'].str.lower().str.contains(mood.lower())]

    # Exclude movies user already watched
    mood_movies = mood_movies[~mood_movies['movie'].isin(user_movies['movie'])]

    recommendations = []
    for row in mood_movies.itertuples():
        pred = predict_rating(user, row.movie)
        recommendations.append((row.movie_name, pred))

    # Sort all movies by predicted rating descending
    recommendations = sorted(recommendations, key=lambda x: x[1], reverse=True)
    return recommendations

# ------------------------------------------------
# 8. Display Recommendations (Pop-up posters, small size)
# ------------------------------------------------
def display_recommendations_all(recommendations, user_id, mood):
    print(f"\n🎬 Recommended Movies for Mood '{mood}' (User {user_id}):\n")
    
    if len(recommendations) == 0:
        print("No movies available for this mood.")
        return
    
    for movie_name, rating in recommendations:
        stars = "⭐" * int(round(rating))
        poster_url, imdb_link = fetch_movie_details(movie_name)

        print(f"{movie_name}\nRating: {rating:.2f} {stars}")
        if imdb_link:
            print(f"IMDb: {imdb_link}")
        print("-" * 60)

        # Show poster in small popup window
        if poster_url and poster_url != "N/A":
            try:
                response = requests.get(poster_url)
                img = Image.open(BytesIO(response.content))
                plt.figure(figsize=(1.5, 2))  # aur chhota poster
                plt.imshow(img)
                plt.axis("off")
                plt.title(movie_name)
                plt.show()
            except:
                pass

# ------------------------------------------------
# 9. Run Program
# ------------------------------------------------
if __name__ == "__main__":
    moods = df['mood'].unique()
    print(f"💡 Available moods: {', '.join(moods)}\n")

    user_id = int(input("Enter your User ID: "))
    mood_input = input("Enter your current mood: ").strip()

    recommendations = recommend_movies_all(user_id, mood_input, df)
    display_recommendations_all(recommendations, user_id, mood_input)
